Notebook to do curved roadway generation

## Changelog
- Friday, Jan 31
    - Creation
    - Toy track generation using way if 10025 from `DR_CHN_Merging_ZS.osm`
- Monday, Feb 3
    - Begin trajdata creation to read in the vehicle tracks

In [ ]:
# usings
using AutomotiveDrivingModels
using AutoViz

In [ ]:
# function: append to curve
function append_to_curve!(target::Curve, newstuff::Curve)
    s_end = target[end].s
    for c in newstuff
        push!(target, CurvePt(c.pos, c.s+s_end, c.k, c.kd))
    end
    return target
end

In [ ]:
# functions: angle calculation
"""
    function append_headings
- Used create the angles and append them into the coordinates

# Examples
```julia
x_coods = [1089.07510, 1093.82626, 1101.19325, 1112.59899, 1123.96733, 1133.24150, 1146.47964]
y_coods = [936.31213, 936.92692,938.52419, 940.93865, 943.27882, 945.21039, 947.88488]
coods = hcat(x_coods,y_coods)
append_headings(coods)
```
"""
function append_headings(coordinates::Matrix{Float64})
    headings = ones(size(coordinates)[1])
    for i = 1:size(coordinates)[1]-1
        # need to make sure that math is right, and that bounds are kept
        tangent_vector = [coordinates[i+1,1]-coordinates[i,1], coordinates[i+1,2]-coordinates[i,2]]
        # @show tangent_vector
        current_heading = get_new_angle(tangent_vector)
        # @show current_heading
        headings[i] = current_heading
    end
    headings[end] = headings[end-1] # assume this is fine
    coordinates = hcat(coordinates, headings)
    return coordinates
end

"""
    function get_new_angle
- Does the actual angle calculation based on the x y coordinates
"""
function get_new_angle(tangent_vector::Array{Float64})
    # it might be a problem when we switch quadrants
    # use signs of tangent vector to get the quadrant of the heading 
    x = tangent_vector[1]
    y = tangent_vector[2]
    if x == 0. && y == 0.
        heading = 0.0
    elseif x == 0.
        heading = π/2 * sign(y) 
    elseif y == 0.
        heading = convert(Float64, π) # this could be either pi or -pi, but just go with pi
    elseif sign(x) == 1 && sign(y) == 1 # first quadrant
        heading = atan(y, x)
    elseif sign(x) == -1 && sign(y) == 1 # second quadrant
        heading = atan(y, x)
    elseif sign(x) == 1 && sign(y) == -1 # fourth quadrant
        heading = atan(y, x)
    elseif sign(x) == -1 && sign(y) == -1 # third quadrant
        heading = atan(y, x)
    end
    # bound_heading doesn't end up getting called cause Julia takes care of it apparently
    bound_heading(heading)

    return heading
end

"""
    function bound_heading
- Make the angle range from 0 to pi instead of going beyond
"""
function bound_heading(heading::Float64)
    if heading > π # send this to be just near -pi
        heading = -π + (heading - π)    # if heading is 3.15, then the new angle will be (-pi + (3.15-pi)) = -3.13
    elseif heading < -π # send this to be just near pi 
        heading = π + (heading + π)     # if heading is -3.15, then the new angle will be (pi + (-3.15+pi)) = 3.13
    end
    return heading
end

In [ ]:
# WBs example code
"""
- WBs example code
"""
function get_track()

    x_coods = [0., 5.]
    y_coods = [3., 3.]
    theta = [0., 0.]
    
    turn_length = 10
    new_theta = 0.
    incremental_angle = (pi/4) / turn_length
    radius = 1
    for i = 1:turn_length
        push!(theta, new_theta)
        push!(x_coods, x_coods[end] + radius * cos(new_theta))
        push!(y_coods, y_coods[end] + radius * sin(new_theta))
        new_theta -= (incremental_angle)
    end
#     plot(x_coods, y_coods)
    mid_coods = vcat(x_coods', y_coods', theta') # x coods in first row, y coods in second row, theta in third row
    
    
    first_cood = VecSE2(mid_coods[1,1], mid_coods[2,1], mid_coods[3,1])
    second_cood = VecSE2(mid_coods[1,2], mid_coods[2,2], mid_coods[3,2])
    radius = 0.01
    nsamples = 20

    track = gen_bezier_curve(first_cood, second_cood, radius, radius, nsamples)
    
    curve_radius = incremental_angle
    nsamples = 1000
    for i = 3:turn_length+2
        turn1 = VecSE2(mid_coods[1, i-1], mid_coods[2, i-1], mid_coods[3, i-1])
        turn2 = VecSE2(mid_coods[1, i], mid_coods[2, i], mid_coods[3, i])
        curve = gen_bezier_curve(turn1, turn2, curve_radius, curve_radius, nsamples)
        append_to_curve!(track, curve)
    end

    return track
end

In [ ]:
# function: get_track
"""
    function get_track()

- Generate a track based on x and y coordinates of the centerline
- Extracts angles as an intermediate step
"""
function get_track()
    # way 10025 for illustrate purposes
    x_coods = [1089.07510, 1093.82626, 1101.19325, 1112.59899, 1123.96733, 1133.24150, 1146.47964]
    y_coods = [936.31213, 936.92692,938.52419, 940.93865, 943.27882, 945.21039, 947.88488]

    coods = hcat(x_coods,y_coods)
    coods_app = append_headings(coods) # Append with the angle
    
    mid_coods = coods_app'
    
    first_cood = VecSE2(mid_coods[1,1], mid_coods[2,1], mid_coods[3,1])
    second_cood = VecSE2(mid_coods[1,2], mid_coods[2,2], mid_coods[3,2])
    radius = 0.01
    nsamples = 20

    track = gen_bezier_curve(first_cood, second_cood, radius, radius, nsamples)
    
    nsamples = 20
    for i = 3:7
        turn1 = VecSE2(mid_coods[1, i-1], mid_coods[2, i-1], mid_coods[3, i-1])
        turn2 = VecSE2(mid_coods[1, i], mid_coods[2, i], mid_coods[3, i])
        curve = gen_bezier_curve(turn1, turn2, radius, radius, nsamples)
        append_to_curve!(track, curve)
    end

    return track
end

In [ ]:
# script: generate the roadway

#nlanes = 1
#length = 50.

#roadway = gen_straight_roadway(nlanes, length)

width = DEFAULT_LANE_WIDTH
roadway = Roadway()

track = get_track()

lane = Lane(LaneTag(1, 2), track, width=DEFAULT_LANE_WIDTH)
push!(roadway.segments, RoadSegment(lane.tag.segment, [lane]))
cam = FitToContentCamera(0.05)
render(roadway, cam=cam)

In [ ]:
@show roadway.segments[1].lanes

## Trajdata experimentation

In [2]:
using DataFrames

In [6]:
using Printf

In [11]:
df = readtable("i101_trajectories-0750am-0805am.txt", separator=' ', header = false);

In [13]:
col_names = [:id, :frame, :n_frames_in_dataset, :epoch, :local_x, :local_y, :global_x, :global_y, :length, :width, :class, :speed, :acc, :lane, :carind_front, :carind_rear, :dist_headway, :time_headway]

18-element Array{Symbol,1}:
 :id                 
 :frame              
 :n_frames_in_dataset
 :epoch              
 :local_x            
 :local_y            
 :global_x           
 :global_y           
 :length             
 :width              
 :class              
 :speed              
 :acc                
 :lane               
 :carind_front       
 :carind_rear        
 :dist_headway       
 :time_headway       

In [14]:
        for (i,name) in enumerate(col_names)
            rename!(df, Symbol(@sprintf("x%d", i)), name)
        end

┌ Warning: `rename!(x::AbstractDataFrame, from::Symbol, to::Symbol)` is deprecated, use `rename!(x, from => to)` instead.
│   caller = top-level scope at In[14]:2
└ @ Core ./In[14]:2


In [16]:
        df[:global_heading] = fill(NaN, nrow(df))

        car2start = Dict{Int, Int}()
        frame2cars = Dict{Int, Vector{Int}}()


Dict{Int64,Array{Int64,1}} with 0 entries

In [18]:
        for (dfind, carid) in enumerate(df[:id])
            if !haskey(car2start, carid)
                car2start[carid] = dfind
            end

            frame = convert(Int, df[dfind, :frame])
            if !haskey(frame2cars, frame)
                frame2cars[frame] = [carid]
            else
                frame2cars[frame] = push!(frame2cars[frame], carid)
            end
        end

In [19]:
car2start

Dict{Int64,Int64} with 2169 entries:
  1704 => 582165
  1760 => 610339
  2843 => 1078311
  306  => 83654
  1090 => 328769
  2111 => 736259
  1333 => 433748
  2381 => 846807
  2812 => 1057790
  1671 => 565979
  1131 => 350079
  2564 => 919923
  2896 => 1108416
  2126 => 741609
  74   => 20386
  1662 => 562965
  2376 => 845294
  905  => 253789
  176  => 48153
  2912 => 1117052
  1469 => 481610
  892  => 250558
  285  => 77433
  1461 => 479439
  2502 => 891904
  ⋮    => ⋮

In [20]:
frame2cars

Dict{Int64,Array{Int64,1}} with 9529 entries:
  2843 => [959, 971, 977, 980, 982, 986, 987, 989, 990, 991  …  1119, 1121, 112…
  4495 => [1454, 1462, 1469, 1470, 1473, 1475, 1477, 1478, 1479, 1481  …  1605,…
  6928 => [2228, 2229, 2230, 2232, 2233, 2235, 2237, 2238, 2239, 2240  …  2393,…
  6380 => [2050, 2053, 2054, 2055, 2057, 2059, 2060, 2061, 2065, 2067  …  2216,…
  1316 => [302, 322, 363, 376, 379, 380, 383, 384, 387, 388  …  554, 555, 558, …
  6265 => [1984, 1989, 1998, 1999, 2001, 2003, 2007, 2008, 2015, 2016  …  2190,…
  5459 => [1731, 1735, 1742, 1748, 1750, 1751, 1752, 1753, 1754, 1755  …  1907,…
  5842 => [1840, 1850, 1867, 1870, 1875, 1876, 1877, 1878, 1879, 1881  …  2054,…
  4473 => [1454, 1456, 1462, 1467, 1468, 1469, 1470, 1471, 1473, 1475  …  1598,…
  5099 => [1607, 1616, 1623, 1631, 1648, 1651, 1654, 1655, 1656, 1657  …  1775,…
  8312 => [2597, 2605, 2608, 2612, 2615, 2618, 2619, 2621, 2623, 2625  …  2880,…
  4460 => [1443, 1447, 1454, 1455, 1456, 1461, 1462, 1467, 1468

## Interaction track experimentation

In [23]:
df = readtable("vehicle_tracks_000.csv", separator=',', header = true)

,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
,Int64⍰,Int64⍰,Int64⍰,String⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1,1,100,car,1067.49,959.049,-6.032,0.215,3.106,4.4,1.77
2,1,2,200,car,1066.89,959.071,-6.035,0.219,3.105,4.4,1.77
3,1,3,300,car,1066.29,959.094,-6.034,0.224,3.105,4.4,1.77
4,1,4,400,car,1065.68,959.118,-6.029,0.228,3.104,4.4,1.77
5,1,5,500,car,1065.08,959.142,-6.019,0.233,3.103,4.4,1.77
6,1,6,600,car,1064.48,959.167,-6.005,0.238,3.102,4.4,1.77
7,1,7,700,car,1063.88,959.192,-5.989,0.244,3.101,4.4,1.77
8,1,8,800,car,1063.28,959.218,-5.971,0.25,3.1,4.4,1.77
9,1,9,900,car,1062.68,959.245,-5.951,0.257,3.098,4.4,1.77


In [24]:
df[:track_id]

108995-element Array{Union{Missing, Int64},1}:
   1
   1
   1
   1
   1
   1
   1
   1
   1
   1
   1
   1
   1
   ⋮
 689
 689
 689
 689
 689
 689
 689
 689
 689
 689
 689
 689

In [32]:
frame2cars = Dict{Int,Vector{Int}}()

Dict{Int64,Array{Int64,1}} with 0 entries

In [34]:
        car2start = Dict{Int, Int}()
        frame2cars = Dict{Int, Vector{Int}}()
for (dfind, carid) in enumerate(df[:track_id])
            if !haskey(car2start, carid)
                car2start[carid] = dfind
            end
    frame = convert(Int, df[dfind, :frame_id])
               if !haskey(frame2cars, frame)
                frame2cars[frame] = [carid]
            else
                frame2cars[frame] = push!(frame2cars[frame], carid)
            end
end

In [35]:
car2start

Dict{Int64,Int64} with 684 entries:
  306 => 44970
  29  => 2324
  74  => 9784
  176 => 26132
  285 => 41721
  318 => 47163
  354 => 52959
  610 => 97390
  563 => 88794
  671 => 107832
  117 => 15978
  284 => 41458
  474 => 71998
  188 => 27769
  589 => 93727
  685 => 108860
  621 => 99323
  617 => 98651
  353 => 52896
  430 => 64733
  79  => 10588
  71  => 8995
  154 => 21943
  184 => 27139
  107 => 14770
  ⋮   => ⋮

In [36]:
frame2cars

Dict{Int64,Array{Int64,1}} with 3512 entries:
  2843 => [480, 483, 488, 490, 496, 500, 506, 511, 518, 528  …  559, 560, 561, …
  1316 => [222, 225, 228, 229, 231, 233, 235, 236, 237, 238  …  246, 247, 248, …
  1333 => [225, 229, 231, 233, 235, 236, 237, 238, 239, 241  …  248, 249, 250, …
  3485 => [627, 636, 638, 642, 643, 648, 654, 656, 658, 659  …  677, 678, 679, …
  3120 => [518, 532, 537, 556, 560, 570, 571, 573, 574, 576  …  615, 616, 617, …
  1671 => [277, 279, 284, 287, 289, 290, 294, 295, 298, 300  …  323, 324, 325, …
  1131 => [179, 186, 188, 195, 197, 199, 201, 203, 204, 206  …  212, 213, 214, …
  74   => [1, 4, 5, 6, 8, 11, 13, 15, 19, 20  …  41, 42, 43, 44, 45, 46, 47, 48…
  1662 => [277, 279, 284, 287, 289, 290, 294, 295, 298, 300  …  320, 321, 322, …
  1265 => [214, 219, 220, 221, 222, 223, 224, 225, 227, 228  …  234, 235, 236, …
  1469 => [250, 251, 256, 259, 261, 262, 264, 265, 266, 268  …  280, 281, 282, …
  2308 => [394, 405, 411, 416, 421, 422, 426, 427, 431, 436  … 

In [37]:
# struct: the interaction trajdata
"""
INTERACTIONTrajdata
The trajectory data stored in the original INTERACTION dataset format.
The dataset is a csv file with columns:
    track_id      - Int64 - Representing the id of the agent
    frame_id      - Int64 - Represents the frames in which the agent appears in the video
    timestamp_ms  - Int64 - represents the time the agent appears in the video. The unit is millisecond
    agent_type    - String - Can be person, car, truck and so on
    x             - Float64 - x position, in meter
    y             - Float64 - y position in meter
    vx            - Float64 - x velocity in m/s
    vy            - Float64 - y velocity in m/s
    psi_rad       - Float64 - yaw angle in radian
    length        - Float64 - Length of the vehicle, in meter
    width         - Float64 - Width of the vehicle, in meter
"""
mutable struct INTERACTIONTrajdata
    df         :: DataFrame
    car2start  :: Dict{Int, Int}         # maps carindex to starting index in the df
    frame2cars :: Dict{Int, Vector{Int}} # maps frame to list of carids in the scene

    function INTERACTIONTrajdata(input_path::String)

        @assert(isfile(input_path))

        df = readtable(input_path, separator=',', header = true)

        car2start = Dict{Int, Int}()
        frame2cars = Dict{Int, Vector{Int}}()

        for (dfind, carid) in enumerate(df[:track_id])
            if !haskey(car2start, carid)
                car2start[carid] = dfind
            end

            frame = convert(Int, df[dfind, :frame_id])
            if !haskey(frame2cars, frame)
                frame2cars[frame] = [carid]
            else
                frame2cars[frame] = push!(frame2cars[frame], carid)
            end
        end

        new(df, car2start, frame2cars)
    end
end

INTERACTIONTrajdata

In [38]:
INTERACTIONTrajdata("vehicle_tracks_000.csv")

INTERACTIONTrajdata(108995×11 DataFrame. Omitted printing of 6 columns
│ Row    │ track_id │ frame_id │ timestamp_ms │ agent_type │ x        │
│        │ Int64⍰   │ Int64⍰   │ Int64⍰       │ String⍰    │ Float64⍰ │
├────────┼──────────┼──────────┼──────────────┼────────────┼──────────┤
│ 1      │ 1        │ 1        │ 100          │ car        │ 1067.49  │
│ 2      │ 1        │ 2        │ 200          │ car        │ 1066.89  │
│ 3      │ 1        │ 3        │ 300          │ car        │ 1066.29  │
│ 4      │ 1        │ 4        │ 400          │ car        │ 1065.68  │
│ 5      │ 1        │ 5        │ 500          │ car        │ 1065.08  │
│ 6      │ 1        │ 6        │ 600          │ car        │ 1064.48  │
│ 7      │ 1        │ 7        │ 700          │ car        │ 1063.88  │
│ 8      │ 1        │ 8        │ 800          │ car        │ 1063.28  │
│ 9      │ 1        │ 9        │ 900          │ car        │ 1062.68  │
│ 10     │ 1        │ 10       │ 1000         │ car        │ 1062